Gradient descent is a very common and simple optimization algorithm, that is often used for model fitting. We can think of this algorithm as traveling along a smooth curved surface to find it's minimums. We start this post by discussing gradient discent in the context of regression for linear systems. The algorithm is completely parameterized by our data points and the selections of

- Model $\;f:\texttt{Observations} \rightarrow \texttt{Prediction}$, is a linear system of equations
- Loss Function

As shown, the implementation complexity is mostly dependant on our choice of model.

# Loss Function and Model

In this first example we select the model $f$ to be a $d$ degree polynomial, specified by it's ordered coefficients. We also select mean squared error as our loss function.

\begin{align*}\texttt{MSE(x)}&=\frac{1}{n}\sum_{i=1}^n\left(\hat{y}_i - f(x_i)\right)^2
\end{align*}

$\texttt{MSE}$ is a good loss function because it is smooth and create a parabola about our target values. These properties allow us to take it's gradient to inform each step of gradient descent.

We note here that gradient descent doesn't garantee that local minima are also the global minimum, but that MSE over a polynomial is a convex function with this desired property.

In [1]:
function MSE{T <: Real}(ps::Matrix{T}, coef::Vector{T})
    # Mean Squared Error
    M = ps[:, 1:end-1]
    y⃗ = ps[:, end]

    mean(sum(([M y⃗]' .* [-1 * coef; 1])', 2) .^ 2)
end

MSE (generic function with 1 method)

# Gradients

We compute the gradient of `MSE(x)` with respect to each **free variable**, defined as our polynomial's coeficients. The gradients tell us how far and in what direction we should move for each **free variable**. As an example, with 2-degree polynomials model our gradients are defined as below.

- Model :: $f(x) = a\cdot x^2 + b\cdot x + c$
- Loss Function :: $\texttt{MSE}$

We can make this a more general solution by computing the gradient as functions $A, K$ of the coefficient's index. 

\begin{align*}
\texttt{L(x;a,b,c)} &=\frac{1}{n}\sum_{i=1}^n\left(\hat{y}_i - a\cdot x_i^2 - b\cdot x_i - c\right)^2\\
K_b(x;a,c) &= \hat{y} - a\cdot x^2 -c\\
A_b(x;b) &= b\cdot x\\
\texttt{L(x;a,b,c)} &=\frac{1}{n}\sum_{i=1}^n\left(K_b(x;a,c) - A_b(x;b)\right)^2\\
&=\frac{1}{n}\sum_{i=1}^n\left(K^2_b(x_i;a,c) -2\cdot K_b(x_i;a,c)\cdot A_b(x_i;b)+ A^2_b(x_i;b)\right)\\
\frac{\partial L}{\partial b} &= \frac{1}{n}\sum_{i=1}^n\left(-2\cdot K_b(x_i;a,c)\frac{\partial A_b}{\partial b} + 2\cdot A_b(x_i;b) \frac{\partial A_b}{\partial b}\right)\\
\frac{\partial A_b}{\partial b} &= x\\
\nabla_b(x) &= \frac{1}{n}\sum_{i=1}^n\left(-2\cdot K_b(x_i;a,c)\cdot x_i + 2\cdot A_b(x_i;b)\cdot x_i\right)\\
\end{align*}

In [2]:
function ∂MSE_∂j{T <: Real}(ps::Matrix{T}, j::Int, coef::Vector{T})
    # Gradient of MSE wrt coeficient at index j | given points
    degree = size(coef, 1)
    target = [convert(Real, (i == j)) for i in 1:degree]
    others = 1.0 .- target

    y⃗ = ps[:, end]
    M = ps[:, 1:end-1]

    vars = [-1 * others .* coef; 1]

    K = sum(([M y⃗]' .* vars)', 2)
    A = [M y⃗][:, j]

    mean((-2 .* K .* A) .+ (2 * (A .^2) .* coef[j]))
end

function expand_poly_2d(X, degree = 3)
    # Model expansion of x for 3rd degree polynomial in 2d
    [r ^ c for r in X, c in 0:degree]
end

expand_poly_2d (generic function with 2 methods)

# Algorithm

The descent algorithm is always the same. With a user defined **learning rate** $\eta$, for each **free variable** at step $s$ :

\begin{align*}
a_{s+1} &= a_{s} - \eta \cdot \nabla_a\\
b_{s+1} &= b_{s} - \eta \cdot \nabla_b\\
c_{s+1} &= c_{s} - \eta \cdot \nabla_c\\
\end{align*}

The **learning rate** describes how big of a jump, along the smooth curve (or more correctly in the direction of the gradient) we should be making. If this value is to large, the `step_gradient` function will repeatedly jump over the target minima. There are exist strategies to prevent this over jumping, like decreasing $\eta$ on every step. In this case, we just select a sufficently small $\eta$.

In [3]:
function step_gradient{T <: Real}(ps::Matrix{T}, coef::Vector{T}, η::T)
    ∇coef::Vector{T} = [∂MSE_∂j(ps, j, coef) for (j, _) in enumerate(coef)]
    Δcoef = map((Δ) -> Δ == NaN ? 0 : Δ, η * ∇coef) # underflow correction
    convert(Vector{T}, [coef[i] - Δ for (i, Δ) in enumerate(Δcoef)])
end

step_gradient (generic function with 1 method)

With all these completed components, gradient descent is now clearly implemented below. 

In [4]:
function gradient_descent{T <: Real}(ps::Matrix{T}, coef::Vector{T}, η::T, steps::Int)
    for i in 1:steps
        coef = step_gradient(ps, coef, η)
    end
    coef
end

gradient_descent (generic function with 1 method)

# Demonstration

Lets look at our solution.

In [5]:
using Plots
pgfplots()

Plots.PGFPlotsBackend()

In [6]:
srand(1337)

poly_from_coeff(coeff) = x -> sum([x^(i-1)*c for (i, c) in enumerate(coeff)])
target(x) = x^3 + 5*x^2 + 7*x + 2

target (generic function with 1 method)

In [15]:
X = collect(linspace(-3.5, 0.5, 70))
Y = [target(x) for x in X]

plt = scatter(X, Y, xlims=(-4, 1), ylims=(-3, 3), lab="observations")

M = hcat([expand_poly_2d(x, 3) for x in X]...)'
Z = convert(Vector{Float64}, rand((size(M, 2),))) # Initial coefficient values
plot!(poly_from_coeff(Z), lab="initial")

@time C = gradient_descent([M Y], Z, .001, 25000)
plot!(poly_from_coeff(C), lab="fitted")

  2.536748 seconds (7.15 M allocations: 2.188 GiB, 14.44% gc time)


<?xml version="1.0" encoding="UTF-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
<path style="stroke:none;" d="M 2.9375 -1.640625 L 2.9375 -0.78125 C 2.9375 -0.421875 2.90625 -0.3125 2.171875 -0.3125 L 1.96875 -0.3125 L 1.96875 0 C 2.375 -0.03125 2.890625 -0.03125 3.3125 -0.03125 C 3.734375 -0.03125 4.25 -0.03125 4.671875 0 L 4.671875 -0.3125 L 4.453125 -0.3125 C 3.71875 -0.3125 3.703125 -0.421875 3.703125 -0.78125 L 3.703125 -1.640625 L 4.6875 -1.640625 L 4.6875 -1.953125 L 3.703125 -1.953125 L 3.703125 -6.484375 C 3.703125 -6.6875 3.703125 -6.75 3.53125 -6.75 C 3.453125 -6.75 3.421875 -6.75 3.34375 -6.625 L 0.28125 -1.953125 L 0.28125 -1.640625 Z M 2.984375 -1.953125 L 0.5625 -1.953125 L 2.984375 -5.671875 Z M 2.984375 -1.953125 "/>
 
 
<path style="stroke:none;" d="M 2.890625 -3.515625 C 3.703125 -3.78125 4.28125 -4.46875 4.28125 -5.265625 C 4.28125 -6.078125 3.40625 -6.640625 2.453125 -6.640625 C 1.453125 -6.640625 0.6875 -6.046875 0.6875 -5.28125 C 0.6875 -4.953125 0.90625 -4.765625 1.203125 -4.765625 C 1.5 -4.765625 1.703125 -4.984375 1.703125 -5.28125 C 1.703125 -5.765625 1.234375 -5.765625 1.09375 -5.765625 C 1.390625 -6.265625 2.046875 -6.390625 2.40625 -6.390625 C 2.828125 -6.390625 3.375 -6.171875 3.375 -5.28125 C 3.375 -5.15625 3.34375 -4.578125 3.09375 -4.140625 C 2.796875 -3.65625 2.453125 -3.625 2.203125 -3.625 C 2.125 -3.609375 1.890625 -3.59375 1.8125 -3.59375 C 1.734375 -3.578125 1.671875 -3.5625 1.671875 -3.46875 C 1.671875 -3.359375 1.734375 -3.359375 1.90625 -3.359375 L 2.34375 -3.359375 C 3.15625 -3.359375 3.53125 -2.6875 3.53125 -1.703125 C 3.53125 -0.34375 2.84375 -0.0625 2.40625 -0.0625 C 1.96875 -0.0625 1.21875 -0.234375 0.875 -0.8125 C 1.21875 -0.765625 1.53125 -0.984375 1.53125 -1.359375 C 1.53125 -1.71875 1.265625 -1.921875 0.984375 -1.921875 C 0.734375 -1.921875 0.421875 -1.78125 0.421875 -1.34375 C 0.421875 -0.4375 1.34375 0.21875 2.4375 0.21875 C 3.65625 0.21875 4.5625 -0.6875 4.5625 -1.703125 C 4.5625 -2.515625 3.921875 -3.296875 2.890625 -3.515625 Z M 2.890625 -3.515625 "/>
 
 
<path style="stroke:none;" d="M 4.46875 -2 C 4.46875 -3.1875 3.65625 -4.1875 2.578125 -4.1875 C 2.109375 -4.1875 1.671875 -4.03125 1.3125 -3.671875 L 1.3125 -5.625 C 1.515625 -5.5625 1.84375 -5.5 2.15625 -5.5 C 3.390625 -5.5 4.09375 -6.40625 4.09375 -6.53125 C 4.09375 -6.59375 4.0625 -6.640625 3.984375 -6.640625 C 3.984375 -6.640625 3.953125 -6.640625 3.90625 -6.609375 C 3.703125 -6.515625 3.21875 -6.3125 2.546875 -6.3125 C 2.15625 -6.3125 1.6875 -6.390625 1.21875 -6.59375 C 1.140625 -6.625 1.125 -6.625 1.109375 -6.625 C 1 -6.625 1 -6.546875 1 -6.390625 L 1 -3.4375 C 1 -3.265625 1 -3.1875 1.140625 -3.1875 C 1.21875 -3.1875 1.234375 -3.203125 1.28125 -3.265625 C 1.390625 -3.421875 1.75 -3.96875 2.5625 -3.96875 C 3.078125 -3.96875 3.328125 -3.515625 3.40625 -3.328125 C 3.5625 -2.953125 3.59375 -2.578125 3.59375 -2.078125 C 3.59375 -1.71875 3.59375 -1.125 3.34375 -0.703125 C 3.109375 -0.3125 2.734375 -0.0625 2.28125 -0.0625 C 1.5625 -0.0625 0.984375 -0.59375 0.8125 -1.171875 C 0.84375 -1.171875 0.875 -1.15625 0.984375 -1.15625 C 1.3125 -1.15625 1.484375 -1.40625 1.484375 -1.640625 C 1.484375 -1.890625 1.3125 -2.140625 0.984375 -2.140625 C 0.84375 -2.140625 0.5 -2.0625 0.5 -1.609375 C 0.5 -0.75 1.1875 0.21875 2.296875 0.21875 C 3.453125 0.21875 4.46875 -0.734375 4.46875 -2 Z M 4.46875 -2 "/>
 
 
<path style="stroke:none;" d="M 1.265625 -0.765625 L 2.328125 -1.796875 C 3.875 -3.171875 4.46875 -3.703125 4.46875 -4.703125 C 4.46875 -5.84375 3.578125 -6.640625 2.359375 -6.640625 C 1.234375 -6.640625 0.5 -5.71875 0.5 -4.828125 C 0.5 -4.28125 1 -4.28125 1.03125 -4.28125 C 1.203125 -4.28125 1.546875 -4.390625 1.546875 -4.8125 C 1.546875 -5.0625 1.359375 -5.328125 1.015625 -5.328125 C 0.9375 -5.328125 0.921875 -5.328125 0.890625 -5.3125 C 1.109375 -5.96875 1.65625 -6.328125 2.234375 -6.328125 C 3.140625 -6.328125 3.5625 -5.515625 3.5625 -4.703125 C 3.5625 -3.90625 3.078125 -3.125 2.515625 -2.5 L 0.609375 -0.375 C 0.5 -0.265625 0.5 -0.234375 0.5 0 

# Higher Dimmensional Data

Although the above example is clear and helpful, it is very rare that we are working with two-dimmensional data. Usually we are attemptying to best fit a model to higher dimmensional predictive affine relationships. Luckly, because we were careful to encapsulate our code in conceptual chunks, we will only need to extend our observation vectors respect our new model. 

- Model :: $f(x, w) = w_0 + \sum_{i=1}^d w_i\cdot x_i + \sum_{i=1}^d\sum_{j=1}^d w_{ij}\cdot x_i \cdot x_j$
- Loss Function :: $\texttt{MSE}$

In [8]:
function take_upper_tri(X::Matrix)
    ret::Vector = []
    for i in 1:size(X, 1)
        for j in i:size(X, 2)
            push!(ret, X[i, j])
        end
    end
    ret
end

function extendx{T <: Real}(x::Vector{T})
    take_upper_tri([a*b for a in x, b in [1; x]])
end

extendx (generic function with 1 method)

# Stochastic Gradient Descent

Unlike gradient descent, which must observe all samples to update on each step, stochastic gradient descent updates on each sample. It is expected that this will have a lower runtime for convergance.

Stocastic gradient descent is expected to converge faster because the direction of the gradient has an opportunity to adjust for every sample, preventing the batch of samples overstepping in one direction.

In [9]:
function stochastic_gradient_descent{T <: Real}(
        ps::Matrix{T}, coef::Vector{T},
        η::T, steps::Int)

    for j in Iterators.take(Iterators.cycle(1:3:size(ps, 1)), steps)
        coef = step_gradient(ps[j:j, :], coef, η)
    end
    coef
end

stochastic_gradient_descent (generic function with 1 method)

In [16]:
plt = scatter(X, Y, xlims=(-4, 1), ylims=(-3, 3), lab="observations")

plot!(poly_from_coeff(Z), lab="initial")

@time C = stochastic_gradient_descent([M Y], Z, .001, 25000)
plot!(poly_from_coeff(C), lab="fitted")

  2.140008 seconds (7.13 M allocations: 355.533 MiB, 7.63% gc time)


<?xml version="1.0" encoding="UTF-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
<path style="stroke:none;" d="M 2.9375 -1.640625 L 2.9375 -0.78125 C 2.9375 -0.421875 2.90625 -0.3125 2.171875 -0.3125 L 1.96875 -0.3125 L 1.96875 0 C 2.375 -0.03125 2.890625 -0.03125 3.3125 -0.03125 C 3.734375 -0.03125 4.25 -0.03125 4.671875 0 L 4.671875 -0.3125 L 4.453125 -0.3125 C 3.71875 -0.3125 3.703125 -0.421875 3.703125 -0.78125 L 3.703125 -1.640625 L 4.6875 -1.640625 L 4.6875 -1.953125 L 3.703125 -1.953125 L 3.703125 -6.484375 C 3.703125 -6.6875 3.703125 -6.75 3.53125 -6.75 C 3.453125 -6.75 3.421875 -6.75 3.34375 -6.625 L 0.28125 -1.953125 L 0.28125 -1.640625 Z M 2.984375 -1.953125 L 0.5625 -1.953125 L 2.984375 -5.671875 Z M 2.984375 -1.953125 "/>
 
 
<path style="stroke:none;" d="M 2.890625 -3.515625 C 3.703125 -3.78125 4.28125 -4.46875 4.28125 -5.265625 C 4.28125 -6.078125 3.40625 -6.640625 2.453125 -6.640625 C 1.453125 -6.640625 0.6875 -6.046875 0.6875 -5.28125 C 0.6875 -4.953125 0.90625 -4.765625 1.203125 -4.765625 C 1.5 -4.765625 1.703125 -4.984375 1.703125 -5.28125 C 1.703125 -5.765625 1.234375 -5.765625 1.09375 -5.765625 C 1.390625 -6.265625 2.046875 -6.390625 2.40625 -6.390625 C 2.828125 -6.390625 3.375 -6.171875 3.375 -5.28125 C 3.375 -5.15625 3.34375 -4.578125 3.09375 -4.140625 C 2.796875 -3.65625 2.453125 -3.625 2.203125 -3.625 C 2.125 -3.609375 1.890625 -3.59375 1.8125 -3.59375 C 1.734375 -3.578125 1.671875 -3.5625 1.671875 -3.46875 C 1.671875 -3.359375 1.734375 -3.359375 1.90625 -3.359375 L 2.34375 -3.359375 C 3.15625 -3.359375 3.53125 -2.6875 3.53125 -1.703125 C 3.53125 -0.34375 2.84375 -0.0625 2.40625 -0.0625 C 1.96875 -0.0625 1.21875 -0.234375 0.875 -0.8125 C 1.21875 -0.765625 1.53125 -0.984375 1.53125 -1.359375 C 1.53125 -1.71875 1.265625 -1.921875 0.984375 -1.921875 C 0.734375 -1.921875 0.421875 -1.78125 0.421875 -1.34375 C 0.421875 -0.4375 1.34375 0.21875 2.4375 0.21875 C 3.65625 0.21875 4.5625 -0.6875 4.5625 -1.703125 C 4.5625 -2.515625 3.921875 -3.296875 2.890625 -3.515625 Z M 2.890625 -3.515625 "/>
 
 
<path style="stroke:none;" d="M 4.46875 -2 C 4.46875 -3.1875 3.65625 -4.1875 2.578125 -4.1875 C 2.109375 -4.1875 1.671875 -4.03125 1.3125 -3.671875 L 1.3125 -5.625 C 1.515625 -5.5625 1.84375 -5.5 2.15625 -5.5 C 3.390625 -5.5 4.09375 -6.40625 4.09375 -6.53125 C 4.09375 -6.59375 4.0625 -6.640625 3.984375 -6.640625 C 3.984375 -6.640625 3.953125 -6.640625 3.90625 -6.609375 C 3.703125 -6.515625 3.21875 -6.3125 2.546875 -6.3125 C 2.15625 -6.3125 1.6875 -6.390625 1.21875 -6.59375 C 1.140625 -6.625 1.125 -6.625 1.109375 -6.625 C 1 -6.625 1 -6.546875 1 -6.390625 L 1 -3.4375 C 1 -3.265625 1 -3.1875 1.140625 -3.1875 C 1.21875 -3.1875 1.234375 -3.203125 1.28125 -3.265625 C 1.390625 -3.421875 1.75 -3.96875 2.5625 -3.96875 C 3.078125 -3.96875 3.328125 -3.515625 3.40625 -3.328125 C 3.5625 -2.953125 3.59375 -2.578125 3.59375 -2.078125 C 3.59375 -1.71875 3.59375 -1.125 3.34375 -0.703125 C 3.109375 -0.3125 2.734375 -0.0625 2.28125 -0.0625 C 1.5625 -0.0625 0.984375 -0.59375 0.8125 -1.171875 C 0.84375 -1.171875 0.875 -1.15625 0.984375 -1.15625 C 1.3125 -1.15625 1.484375 -1.40625 1.484375 -1.640625 C 1.484375 -1.890625 1.3125 -2.140625 0.984375 -2.140625 C 0.84375 -2.140625 0.5 -2.0625 0.5 -1.609375 C 0.5 -0.75 1.1875 0.21875 2.296875 0.21875 C 3.453125 0.21875 4.46875 -0.734375 4.46875 -2 Z M 4.46875 -2 "/>
 
 
<path style="stroke:none;" d="M 1.265625 -0.765625 L 2.328125 -1.796875 C 3.875 -3.171875 4.46875 -3.703125 4.46875 -4.703125 C 4.46875 -5.84375 3.578125 -6.640625 2.359375 -6.640625 C 1.234375 -6.640625 0.5 -5.71875 0.5 -4.828125 C 0.5 -4.28125 1 -4.28125 1.03125 -4.28125 C 1.203125 -4.28125 1.546875 -4.390625 1.546875 -4.8125 C 1.546875 -5.0625 1.359375 -5.328125 1.015625 -5.328125 C 0.9375 -5.328125 0.921875 -5.328125 0.890625 -5.3125 C 1.109375 -5.96875 1.65625 -6.328125 2.234375 -6.328125 C 3.140625 -6.328125 3.5625 -5.515625 3.5625 -4.703125 C 3.5625 -3.90625 3.078125 -3.125 2.515625 -2.5 L 0.609375 -0.375 C 0.5 -0.265625 0.5 -0.234375 0.5 0 